# ARIMAモデル，1次式型トレンドを含むデータ

1. ARMAシステム（真）のデータに1次式データ（非定常）が重畳する。<br>
2. ARIMAモデルの適用<br>
3. カーブフィッティングで１次式型トレンドを除去後にARMAモデルを適用<br>


### pandas データフレームを使う
- pandas.date_range https://pandas.pydata.org/docs/reference/api/pandas.date_range.html
- freqの説明，"Offset aliases"  https://pandas.pydata.org/docs/user_guide/timeseries.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
#import pandas as pd
from scipy import stats

import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima_process import arma_impulse_response
from statsmodels.graphics.tsaplots import plot_predict

import matplotlib.pyplot as plt
%matplotlib inline

観測データ（トレニーングデータ）              y  
予測精度を見るための実データ（テストデータ）  y_test

In [ ]:
np.random.seed(123)

ar = [1, -1.5, 0.7]
ma = [1.0, 0.6]

N, Ntest = 1000, 100
Nall = N + Ntest

xindex = np.arange(Nall)
input = lambda size: np.random.normal(loc=0.0, scale=1.0, size=size)  # input for ARMA

# ARMAモデルの入力 = scale*input,  出力 y0_all
y0_all = arma_generate_sample(ar=ar, ma=ma, nsample=Nall, scale=1, distrvs=input, burnin=500)


# トレンドの信号（1次式型）
coef_a, coef_b = 0.05, 4
trend_all = coef_a*np.arange(Nall) + coef_b

# 出力信号（トレーニングデータ＋テストデータ）
y_all = y0_all + trend_all

#信号だけのデータを作成
y = y_all[:N]      #観測データはｙ
y_test = y_all[N:] #テストデータ，プロット用で使用

fig, ax = plt.subplots(figsize=(6,3))
plt.plot(xindex[:N], y, c='k', label='y')
plt.plot(xindex[N:], y_test, c='b', label='y_test')
plt.plot(y0_all, c='gray', linestyle='dashed', label='y0_all')
plt.legend()
plt.grid()

#plt.savefig('fig_ARIMA_ident_signal_trend.png', bbox_inches='tight')
plt.show()

#### ARIMA 同定 ARIMA(2,1,1)

In [ ]:
arima211 = ARIMA(y, order=(2,1,1), trend='n').fit()
print(arima211.summary())

#### 残差の検証

In [ ]:
arima211.plot_diagnostics(figsize=(10,6))
#plt.savefig('fig_ARIMA_diag.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#　注意：グラフ横軸の0はxstartに相当
#xstart, xend = N-100, N+30
xstart, xend = N-600, N+30

fig, ax = plt.subplots(figsize=(7,3))

plot_predict(arima211, start=xstart, end=xend, ax=ax, dynamic=False)
ax.plot(y_all[xstart:xend], c='k', linestyle='dashed', label='y_all') # y
ax.legend(loc='upper left')

#plt.savefig('fig_ARIMA_predict.pdf', bbox_inches='tight')
plt.show()

#### ARIMA 同定 ARIMA(2, 2, 1)

In [ ]:
arima221 = ARIMA(y, order=(2,2,1), trend='n').fit()
print(arima221.summary())

In [ ]:
arima221.plot_diagnostics(figsize=(10,6))
#plt.savefig('fig_ARIMA_diag02.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7,3))

plot_predict(arima221, start=xstart, end=xend, ax=ax, dynamic=False)
ax.plot(y_all[xstart:xend], c='k', linestyle='dashed', label='y_all') # y
ax.legend(loc='upper left')

#plt.savefig('fig_ARIMA_predict02.pdf', bbox_inches='tight')
plt.show()

In [ ]:
arima222 = ARIMA(y, order=(2,2,2), trend='n').fit()
print(arima222.summary())

### カーブフィッティング法  
1次式トレンドを除去する方法

In [ ]:
from scipy.optimize import curve_fit

def fun_linear(x, a, b):
    return a*x + b

In [ ]:
parm, hmat = curve_fit(fun_linear, xindex[:N], y)
y_mod = y - (parm[0]*xindex[:N] + parm[1])
print(parm)

#### y0_all と y_modのプロット
- y0_all : トレンドを重畳させる前のデータ
- y_mod : yからトレンドを除去したデータ

両者をプロットすると，ほぼ，一致しているように見える。

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
plt.plot(xindex[:N], y0_all[:N], c='k', label='y')
plt.plot(xindex[:N], y_mod, c='b', linestyle='dashed', label='y_mod')
plt.legend()
plt.grid()

#plt.savefig('fig_ARIMA_ident_signal_trend.png')
plt.show()

In [ ]:
arma21 = ARIMA(y_mod, order=(2,0,1), trend='n').fit()
print(arma21.summary())